## visualize images

In [13]:
FileList = glob.glob('F:/pfe_2/realsense/frames/rgb/*.npy')
i = 0
bar = progressbar.ProgressBar(maxval=len(FileList), \
            widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
for imagef in FileList:
    image = np.load(imagef)
    cv2.imwrite('rgb/'+str(i).zfill(3)+'.jpg', image)
    bar.update(i+1)
    i += 1
bar.finish()

[========================================================================] 100%


![title](img/imgs.PNG)

## 1. Loop closure detection test

In [1]:
from LoopClosure import LoopClosure
import glob
import numpy as np
import cv2
import progressbar

### 1.1 init by training

In [7]:
# init class
loopclosure = LoopClosure()

FileList = glob.glob('F:/pfe_2/realsense/frames/rgb/*.npy')
# train the bag of words by the images of the file list
loopclosure.set_vocabulary_by_train(FileList)
# save the vocabulary
loopclosure.save_vocabulary('voca.npy')

--- start training ---
--- train finished ---


### 1.2 init by load trained BOW

In [2]:
loopclosure = LoopClosure()
loopclosure.set_vocabulary('voca.npy')

### 1.3 test

In [3]:
# to reset the image signatures
loopclosure.image_signatures = []
for i in range(150):
    image = np.load('F:/pfe_2/realsense/frames/rgb/'+str(i).zfill(3)+'.npy')
    inp = loopclosure.get_signature(image)
    
    d, n = loopclosure.loop_closure_detection(inp, i, 0.07, 20)
    #print(i,'if detected loop:', d, ',   if true with image num: ',n )
    if d :
        print(i,'if detected loop:', d, ',   if true with image num: ',n )

67 if detected loop: True ,   if true with image num:  46
69 if detected loop: True ,   if true with image num:  48
70 if detected loop: True ,   if true with image num:  46
71 if detected loop: True ,   if true with image num:  47
72 if detected loop: True ,   if true with image num:  49
73 if detected loop: True ,   if true with image num:  51
140 if detected loop: True ,   if true with image num:  35
141 if detected loop: True ,   if true with image num:  35
142 if detected loop: True ,   if true with image num:  35
145 if detected loop: True ,   if true with image num:  35
146 if detected loop: True ,   if true with image num:  35
147 if detected loop: True ,   if true with image num:  35
148 if detected loop: True ,   if true with image num:  35


## 2. Normal SLAM test
this is the tracking part of the slam

In [1]:
from IdvSlam import IdvSlam
from pathlib import Path
import os

In [15]:
doc = 'F:/pfe_2/realsense/frames'
# doc is the dictonary of the input images
# and the second input for the class is '', which is the location to create the output files
slam = IdvSlam(doc, '')

In [16]:
# set the parameter of feature marching
slam.set_match_param(0.7)
# reset the slam (reset the tracking camera position)
slam.reset_Ms()
# calculate from the 50th image and for 150 images
slam.calculate_for_n_images(150, 50)

[========================================================================] 100%


0

In [17]:
# make the points cloud ply file for visualize
slam.make_ply(10)

[========================================================================] 100%


slam result with out loop closing

beginning from the entrance door (blue cycle) and end with the same door (in red cycle), but because of the accumaltion of error, the door at the beginning and at the end are seperated.


![title](img/non_cl_1.PNG)

## 3. Loop closure Slam test

In [3]:
from IdvLoopSlam import IdvLoopSlam
import glob
import numpy as np
import matplotlib.pyplot as plt
import winsound
duration = 1000  # millisecond
freq = 440  # Hz

### define with training

In [19]:
doc = 'F:/pfe_2/realsense/frames'
slam = IdvLoopSlam(doc, '')
FileList = glob.glob('F:/pfe_2/realsense/frames/rgb/*.npy')
slam.set_vocabulary_by_train(FileList)
file_dictionary = 'voca.npy'
slam.loopclosure.save_vocabulary(file_dictionary)

--- start training ---
--- train finished ---


### define with old trained

In [4]:
doc = 'F:/pfe_2/realsense/frames'
slam = IdvLoopSlam(doc, '')
slam.set_vocabulary('voca.npy')

### main process

In [5]:
# set the parameter of feature matching
slam.set_match_param(0.7)
slam.reset_Ms()
# set the maximum image of refine when detecting loop
slam.set_max_image(100)
# set the range for not take in to count when detect loop closure
slam.set_loop_detect_range(20)
# calculate from the 50th image and for 150 images
slam.calculate_for_n_images(150, 50)
# draw all the camera position
slam.draw_camera()

128 image correction of matrix with image 5: ◢                                 
129 image correction of matrix with image 5: ◢                                 
133 image correction of matrix with image 10: ◢                                
136 image correction of matrix with image 13: ◢                                
137 image correction of matrix with image 13: ◢                                
138 image correction of matrix with image 14: ◢                                
139 image correction of matrix with image 15: ◢                                
140 image correction of matrix with image 17: ◢                                
141 image correction of matrix with image 18: ◢                                
142 image correction of matrix with image 17: ◢                                
143 image correction of matrix with image 17: ◢                                
144 image correction of matrix with image 19: ◢                                
145 image correction of matrix with imag

 --- camera drawn


0

In [4]:
slam.make_ply(10) 
#winsound.Beep(freq, duration)

[========================================================================] 100%


the result with loop closure
![title](img/pts_cl.PNG)

the loop is closed compared with the upper result.
as the "two" doors become one single door (same for the sofas)
![title](img/with_cl.PNG)

## 4.  Mapping test (with segmentation result)

In [1]:
from Mapping import Mapping
import numpy as np
import pickle

In [2]:
doc = 'F:/pfe_2/realsense/frames'
Ms = np.load('IdvSlam/matrix.npy')
mapping = Mapping(doc, '')

In [3]:
mapping.reset()
# imgid is the ids for the image with segmentation outputs
imgid = np.array([50,62,69,79,109,119,149,159,169,179,189,198])
for i in range(len(imgid)):
    idx = imgid[i]-50
    mapping.add_keyframe(idx , Ms[idx])

In [4]:
mapping.draw_Map()

version 1.0 

the points of the objects and the bounding box of these objects
![title](img/obj_bb.PNG)

## 5. Test with updated continue slam

new version of slam

which take one image one time as input,
and can create multi loops and detect loop closure to connect these loops (as in real using, the some image without much features may interrupt the whole process).
![title](img/slam.jpg)

In [ ]:
from IdvLoopSlam import IdvLoopSlam
import glob
import numpy as np
import matplotlib.pyplot as plt
import winsound
duration = 1000  # millisecond
freq = 440  # Hz

In [ ]:
doc = 'F:/pfe_2/realsense/frames'
slam = IdvLoopSlam(doc, '')
slam.set_vocabulary('voca.npy')

In [ ]:
# set the parameter of feature matching
slam.set_match_param(0.7)
slam.reset_Ms()
# set the range for not take in to count when detect loop closure
slam.set_loop_detect_range(20)
for True:
    image = 
    depth = 
    slam.get_one_keyframe(image, depth)
    if :
        break
# draw all the camera position
slam.draw_camera()
slam.save_for_mapping()